In [ ]:
import time
import taskingai
# Load TaskingAI API Key from environment variable

# TaskingAI: Chat with Assistant Example

In this example, we will first create an assistant who knows the meaning of various numbers and will explain it in certain language.
Then we will start a chat with the assistant.

##  Create Assistant

In [ ]:
from taskingai.tool import Action, ActionAuthentication, ActionAuthenticationType
from typing import List

# create an assistant action
NUMBERS_API_SCHEMA = {
        "openapi": "3.0.0",
        "info": {
            "title": "Numbers API",
            "version": "1.0.0",
            "description": "API for fetching interesting number facts"
        },
        "servers": [
            {
            "url": "http://numbersapi.com"
            }
        ],
        "paths": {
            "/{number}": {
            "get": {
                "description": "Get a fact about a number",
                "operationId": "getNumberFact",
                "parameters": [
                {
                    "name": "number",
                    "in": "path",
                    "required": True,
                    "description": "The number to get the fact for",
                    "schema": {
                    "type": "integer"
                    }
                }
                ],
                "responses": {
                "200": {
                    "description": "A fact about the number",
                    "content": {
                    "text/plain": {
                        "schema": {
                        "type": "string"
                        }
                    }
                    }
                }
                }
            }
        }
    }
}
actions: List[Action] = taskingai.tool.bulk_create_actions(
    schema=NUMBERS_API_SCHEMA,
    authentication=ActionAuthentication(
        type=ActionAuthenticationType.NONE,
    )
)
action = actions[0]
print(f"created action: {action}\n")

In [ ]:
from taskingai.assistant import Assistant, Chat, AssistantTool, AssistantToolType
from taskingai.assistant.memory import AssistantMessageWindowMemory

# choose an available chat_completion model from your project
model_id = "YOUR_MODEL_ID"

assistant: Assistant = taskingai.assistant.create_assistant(
    model_id=model_id,
    name="My Assistant",
    description="A assistant who knows the meaning of various numbers.",
    memory=AssistantMessageWindowMemory(
        max_messages=20,
        max_tokens=1000
    ),
    system_prompt_template=[
        "You know the meaning of various numbers.",
        "No matter what the user's language is, you will use the {{langugae}} to explain."
    ],
    tools=[AssistantTool(
        type=AssistantToolType.action,
        id=action.action_id,
    )],
    retrievals=[],
    metadata={"foo": "bar"},
)
print(f"created assistant: {assistant}\n")

## Start a Chat 

In [ ]:
chat: Chat = taskingai.assistant.create_chat(
    assistant_id=assistant.assistant_id,
)
print(f"created chat: {chat.chat_id}\n")

In [ ]:
from taskingai.assistant import Message, MessageChunk
user_input = input("User Input: ")
while user_input.strip() and user_input != "q":
    # create user message
    taskingai.assistant.create_message(
        assistant_id=assistant.assistant_id,
        chat_id=chat.chat_id,
        text=user_input,
    )
    print(f"User: {user_input}")
    
    # generate assistant response
    assistant_message: Message = taskingai.assistant.generate_message(
        assistant_id=assistant.assistant_id,
        chat_id=chat.chat_id,
        system_prompt_variables={
            "language": "English"
        }
    )
    print(f"Assistant: {assistant_message.content.text}")
    time.sleep(2)
    # quit by input 'q
    user_input = input("User: ")

In [ ]:
user_input = input("User Input: ")
while user_input.strip() and user_input != "q":
    # create user message
    taskingai.assistant.create_message(
        assistant_id=assistant.assistant_id,
        chat_id=chat.chat_id,
        text=user_input,
    )
    print(f"User: {user_input}")
    
    # generate assistant response
    assistant_message_response = taskingai.assistant.generate_message(
        assistant_id=assistant.assistant_id,
        chat_id=chat.chat_id,
        system_prompt_variables={
            "language": "English"
        },
        stream=True,
    )
    
    print(f"Assistant:", end=" ", flush=True)
    for item in assistant_message_response:
        if isinstance(item, MessageChunk):
            print(item.delta, end="", flush=True)
        elif isinstance(item, Message):
            print(f"\nmessage_id: {item.message_id}")
            
    time.sleep(2)
    # quit by input 'q
    user_input = input("User: ")

In [ ]:
# list messages
messages = taskingai.assistant.list_messages(
    assistant_id=assistant.assistant_id,
    chat_id=chat.chat_id,
)
for message in messages:
    print(f"{message.role}: {message.content.text}")

In [ ]:
# delete assistant
taskingai.assistant.delete_assistant(
    assistant_id=assistant.assistant_id,
)

In [ ]:
# list messages
messages = taskingai.assistant.list_messages(
    assistant_id=assistant.assistant_id,
    chat_id=chat.chat_id,
)
for message in messages:
    print(f"{message.role}: {message.content.text}")